## Оптимизация выполнения кода, векторизация, Numba

## Лабораторная работа 8

In [28]:
pip install line_profiler

Note: you may need to restart the kernel to use updated packages.


In [77]:
import numpy as np
import pandas as pd
from numba import jit, njit, vectorize, float64

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [2]:
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
reviews = pd.read_csv('reviews_sample.csv', index_col=0, parse_dates=['date'])

In [7]:
recipes.head(3)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0


In [8]:
reviews.head(3)

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


In [38]:
# A
def get_mean_rating_of_2010_A(df):
    sum_, k = 0, 0
    
    for index_, row_ in df.iterrows():
        if row_['date'].year == 2010:
            sum_ += row_['rating']
            k += 1
    
    return sum_ / k


# B
def get_mean_rating_of_2010_B(df):
    sum_, k = 0, 0
    df = df[df['date'].dt.year == 2010]
    
    for index_, row_ in df.iterrows():
        sum_ += row_['rating']
        k += 1
    
    return sum_ / k


# B1
def get_mean_rating_of_2010_B1(df):
    rating_ = df[df['date'].dt.year == 2010]['rating']
    sum_ = sum(df['rating'])
    
    return sum_ / len(rating_)

# C
def get_mean_rating_of_2010_C(df):
    return df[df['date'].dt.year == 2010]['rating'].mean()

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [20]:
%timeit -r 2 get_mean_rating_of_2010_A(reviews)

17.2 s ± 108 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [21]:
%timeit -r 2 get_mean_rating_of_2010_B(reviews)

1.63 s ± 4.4 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [22]:
%timeit -r 2 get_mean_rating_of_2010_B1(reviews)

57.1 ms ± 301 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [23]:
%timeit -r 2 get_mean_rating_of_2010_C(reviews)

31 ms ± 179 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [39]:
%lprun -f get_mean_rating_of_2010_B get_mean_rating_of_2010_B(reviews)

Timer unit: 1e-07 s

Total time: 5.68894 s
File: C:\Users\Alice Klimovich\AppData\Local\Temp\ipykernel_17780\1568868108.py
Function: get_mean_rating_of_2010_B at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def get_mean_rating_of_2010_B(df):
    15         1         12.0     12.0      0.0      sum_, k = 0, 0
    16         1     273292.0 273292.0      0.5      df = df[df['date'].dt.year == 2010]
    17                                               
    18     12095   49370149.0   4081.9     86.8      for index_, row_ in df.iterrows():
    19     12094    6872144.0    568.2     12.1          sum_ += row_['rating']
    20     12094     373826.0     30.9      0.7          k += 1
    21                                               
    22         1         20.0     20.0      0.0      return sum_ / k

Итак, большая часть времени выполнения функции уходит на цикл (почти 90 %)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [42]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [48]:
from collections import defaultdict

def get_word_reviews_count_B(df):
    word_reviews_count = defaultdict(int)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        
        for word in row['review'].split(' '):
            word_reviews_count[word] += 1
    
    return word_reviews_count

In [44]:
from collections import Counter

def get_word_reviews_count_C(df):
    words = reviews['review'].dropna().str.split(' ').explode()
    return Counter(words)

In [45]:
%%time
get_word_reviews_count(reviews)

CPU times: total: 54.7 s
Wall time: 55.1 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [49]:
%%time
get_word_reviews_count_B(reviews)

CPU times: total: 27.5 s
Wall time: 27.8 s


defaultdict(int,
            {'Last': 94,
             'week': 804,
             'whole': 5628,
             'sides': 312,
             'of': 109029,
             'frozen': 2647,
             'salmon': 729,
             'fillet': 60,
             'was': 88781,
             'on': 34583,
             'sale': 149,
             'in': 61539,
             'my': 44144,
             'local': 561,
             'supermarket,': 10,
             'so': 46090,
             'I': 285147,
             'bought': 1369,
             'tons': 161,
             '(okay,': 5,
             'only': 13965,
             '3,': 48,
             'but': 42513,
             'total': 381,
             'weight': 160,
             'over': 9065,
             '10': 2303,
             'pounds).': 2,
             '': 214145,
             'This': 39448,
             'recipe': 41098,
             'is': 55075,
             'perfect': 4398,
             'for': 121224,
             'fillet,': 14,
             'even': 7878,
       

In [47]:
%%time
get_word_reviews_count_C(reviews)

CPU times: total: 6.78 s
Wall time: 6.9 s


Counter({'I': 285147,
         'the': 266050,
         'and': 217849,
         '': 214145,
         'a': 166136,
         'to': 131471,
         'for': 121224,
         'it': 111175,
         'of': 109029,
         'was': 88781,
         'this': 87057,
         'in': 61539,
         'with': 58374,
         'is': 55075,
         'so': 46090,
         'used': 44849,
         'my': 44144,
         'but': 42513,
         'recipe': 41098,
         'This': 39448,
         'made': 38087,
         'that': 36984,
         'as': 35493,
         'have': 35190,
         'on': 34583,
         'make': 32124,
         'Thanks': 26556,
         'had': 26362,
         'The': 25720,
         'very': 25681,
         'be': 25032,
         'just': 24944,
         'you': 24475,
         'will': 23985,
         'out': 23644,
         'not': 22907,
         'added': 21710,
         'some': 20753,
         'were': 20716,
         'like': 20171,
         'them': 19689,
         'It': 19417,
         'really': 1

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [6]:
mean_rating = reviews.groupby('recipe_id')['rating'].mean().rename('mean_rating')
reviews_with_mean = reviews.merge(mean_rating, left_on='recipe_id', right_index=True)
reviews_with_mean = reviews_with_mean.query("rating > 0")
reviews_with_mean

,user_id,recipe_id,date,rating,review,mean_rating
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,4.818182
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,4.583333
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",4.000000
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,3.727273
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,5.000000
...,...,...,...,...,...,...
344676,724631,314698,2008-10-14,5,"5 Stars, The Hunter said the mushrooms and sau...",5.000000
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...,3.833333
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou...",5.000000
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...,3.705882


In [83]:
def MAPE_A(df):
    # Словарь для хранения MAPE для каждого рецепта
    mape_per_recipe = {}

    # Получаем уникальные recipe_id
    unique_recipe_ids = df['recipe_id'].unique()

    for recipe_id in unique_recipe_ids:
        # Фильтруем строки для текущего recipe_id
        recipe_rows = df[df['recipe_id'] == recipe_id]

        mape_total = 0
        n = len(recipe_rows)

        for i in range(n):
            rating = recipe_rows.iloc[i]['rating']
            mean_rating = recipe_rows.iloc[i]['mean_rating']
            mape_total += abs((rating - mean_rating) / mean_rating)

        # Вычисляем MAPE для текущего рецепта и добавляем его в словарь
        mape = (mape_total / n) * 100
        mape_per_recipe[recipe_id] = mape

    return mape_per_recipe

In [84]:
%%time
MAPE_A(reviews_with_mean)

CPU times: total: 1min 17s
Wall time: 1min 17s


{57993: 6.861063464837047,
 142201: 9.090909090909097,
 252013: 0.0,
 404716: 38.48238482384824,
 129396: 0.0,
 31322: 11.11111111111111,
 199579: 12.134502923976576,
 16067: 4.891171435558819,
 33715: 0.0,
 11252: 17.33333333333333,
 154996: 7.6923076923076845,
 7974: 11.111111111111112,
 150499: 13.749999999999998,
 114415: 0.0,
 21171: 7.427055702917763,
 49200: 11.654000205402063,
 51209: 14.23925339366518,
 320714: 9.523809523809518,
 38249: 22.38805970149253,
 213395: 0.0,
 171615: 0.0,
 122289: 13.368983957219251,
 148347: 15.131578947368418,
 328668: 9.99999999999999,
 42603: 13.444207533759775,
 300681: 0.0,
 299881: 5.747126436781613,
 61610: 10.403968816442244,
 44010: 0.0,
 105102: 9.595287858635778,
 352053: 49.99999999999999,
 57408: 0.0,
 90485: 19.58041958041958,
 75168: 4.748858447488591,
 52035: 11.37625244731088,
 277781: 0.0,
 280223: 12.757201646090538,
 53503: 9.374999999999991,
 13707: 16.345029239766088,
 39468: 14.814814814814813,
 291234: 7.235142118863052,
 1

In [10]:
@njit
def MAPE_B(recipe_ids, ratings, mean_ratings):
    unique_recipe_ids = list(set(recipe_ids))
    mape_per_recipe = {}

    for recipe_id in unique_recipe_ids:
        mape_total = 0
        n = 0
        
        for i in range(len(recipe_ids)):
            if recipe_ids[i] == recipe_id:
                rating = ratings[i]
                mean_rating = mean_ratings[i]
                mape_total += abs((rating - mean_rating) / mean_rating)
                n += 1
        
        mape = (mape_total / n) * 100
        mape_per_recipe[recipe_id] = mape

    return mape_per_recipe

In [11]:
recipe_ids = reviews_with_mean['recipe_id'].values
ratings = reviews_with_mean['rating'].values
mean_ratings = reviews_with_mean['mean_rating'].values

In [15]:
%%time
MAPE_B(recipe_ids, ratings, mean_ratings)

CPU times: total: 3.83 s
Wall time: 3.82 s


DictType[int64,float64]<iv=None>({262144: 0.0, 524289: 0.0, 262166: 0.0, 262188: 0.0, 48: 100.0, 55: 7.894736842105263, 262207: 0.0, 66: 2.122347066167288, 262214: 16.666666666666664, 262219: 0.0, 91: 7.894736842105263, 94: 0.0, 128: 0.0, 262285: 20.512820512820518, 262293: 0.0, 153: 25.806451612903224, 524456: 0.0, 176: 0.0, 181: 41.66666666666667, 262325: 0.0, 262327: 7.894736842105263, 186: 7.235142118863049, 262330: 44.44444444444444, 524487: 0.0, 203: 20.377867746288807, 262348: 16.666666666666664, 524495: 0.0, 224: 66.66666666666666, 240: 16.666666666666664, 262386: 0.0, 246: 12.444444444444448, 262400: 0.0, 282: 0.0, 284: 0.0, 288: 50.000000000000014, 289: 0.0, 314: 6.174957118353343, 318: 7.894736842105263, 321: 0.0, 337: 0.0, 344: 0.0, 352: 33.33333333333333, 360: 0.0, 373: 0.0, 378: 8.468468468468473, 379: 9.523809523809518, 381: 0.0, 383: 0.0, 385: 0.0, 524675: 0.0, 387: 0.0, 401: 0.0, 262550: 0.0, 262563: 0.0, 262564: 0.0, 524716: 0.0, 262577: 0.0, 262585: 50.0, 445: 14.344

In [30]:
def MAPE_C(df):
    def compute_mape(ratings, mean_ratings):
        return np.mean(np.abs((ratings - mean_ratings) / mean_ratings)) * 100

    grouped = df.groupby('recipe_id')
    mape_per_recipe = {recipe_id: compute_mape(group['rating'].values, group['mean_rating'].values) for recipe_id, group in grouped}
    return mape_per_recipe

In [31]:
%%time
MAPE_C(reviews_with_mean)

CPU times: total: 7.58 s
Wall time: 7.61 s


{48: 100.0,
 55: 7.894736842105263,
 66: 2.122347066167287,
 91: 7.894736842105263,
 94: 0.0,
 128: 0.0,
 153: 25.80645161290323,
 176: 0.0,
 181: 41.66666666666667,
 186: 7.23514211886305,
 203: 20.377867746288807,
 224: 66.66666666666666,
 240: 16.666666666666664,
 246: 12.444444444444445,
 282: 0.0,
 284: 0.0,
 288: 50.000000000000014,
 289: 0.0,
 314: 6.174957118353343,
 318: 7.894736842105263,
 321: 0.0,
 337: 0.0,
 344: 0.0,
 352: 33.33333333333333,
 360: 0.0,
 373: 0.0,
 378: 8.46846846846847,
 379: 9.523809523809518,
 381: 0.0,
 383: 0.0,
 385: 0.0,
 387: 0.0,
 401: 0.0,
 445: 14.344827586206893,
 465: 0.0,
 469: 25.000000000000007,
 502: 0.0,
 504: 0.0,
 544: 11.059907834101383,
 546: 9.523809523809518,
 561: 5.747126436781613,
 587: 0.0,
 635: 0.0,
 651: 11.11111111111111,
 653: 0.0,
 658: 0.0,
 659: 16.666666666666664,
 671: 9.609609609609606,
 692: 0.0,
 712: 0.0,
 719: 17.307692307692307,
 779: 0.0,
 884: 0.0,
 916: 26.76005747126437,
 953: 0.0,
 965: 8.658008658008656,
 9

In [34]:
#С np.vectorize (медленнее)

def MAPE_C(df):
    def compute_mape(ratings, mean_ratings):
        return np.mean(np.abs((ratings - mean_ratings) / mean_ratings)) * 100
    
    # Векторизуем функцию compute_mape
    vectorized_compute_mape = np.vectorize(compute_mape)
    
    grouped = df.groupby('recipe_id')
    
    mape_per_recipe = {recipe_id: vectorized_compute_mape(group['rating'].values, group['mean_rating'].values)
                       for recipe_id, group in grouped}
    
    return mape_per_recipe

In [35]:
%%time
MAPE_C(reviews_with_mean)

CPU times: total: 12.9 s
Wall time: 12.9 s


{48: array([100.]),
 55: array([ 5.26315789,  5.26315789, 15.78947368,  5.26315789]),
 66: array([ 1.12359551,  1.12359551,  1.12359551,  1.12359551, 19.1011236 ,
         1.12359551,  1.12359551,  1.12359551,  1.12359551,  1.12359551,
         1.12359551,  1.12359551,  1.12359551,  1.12359551,  1.12359551,
         1.12359551,  1.12359551,  1.12359551]),
 91: array([ 5.26315789,  5.26315789, 15.78947368,  5.26315789]),
 94: array([0., 0., 0., 0.]),
 128: array([0., 0.]),
 153: array([27.45098039, 27.45098039, 27.45098039,  1.96078431, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
         1.96078431, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039]),
 176: array([0.]),
 181: array([50. , 62.5, 12.5]),
 186:

In [65]:
# С apply (тоже медленее)

def MAPE_C(df):
    def compute_mape(group):
        ratings = group['rating'].values
        mean_ratings = group['mean_rating'].values
        return np.mean(np.abs((ratings - mean_ratings) / mean_ratings)) * 100

    # Применяем compute_mape к каждой группе
    grouped = df.groupby('recipe_id').apply(compute_mape)
    
    return grouped.to_dict()

In [66]:
%%time
MAPE_C(reviews_with_mean)

CPU times: total: 8.66 s
Wall time: 8.71 s


C:\Users\Alice Klimovich\AppData\Local\Temp\ipykernel_20556\4049238575.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby('recipe_id').apply(compute_mape)


{48: 100.0,
 55: 7.894736842105263,
 66: 2.122347066167287,
 91: 7.894736842105263,
 94: 0.0,
 128: 0.0,
 153: 25.80645161290323,
 176: 0.0,
 181: 41.66666666666667,
 186: 7.23514211886305,
 203: 20.377867746288807,
 224: 66.66666666666666,
 240: 16.666666666666664,
 246: 12.444444444444445,
 282: 0.0,
 284: 0.0,
 288: 50.000000000000014,
 289: 0.0,
 314: 6.174957118353343,
 318: 7.894736842105263,
 321: 0.0,
 337: 0.0,
 344: 0.0,
 352: 33.33333333333333,
 360: 0.0,
 373: 0.0,
 378: 8.46846846846847,
 379: 9.523809523809518,
 381: 0.0,
 383: 0.0,
 385: 0.0,
 387: 0.0,
 401: 0.0,
 445: 14.344827586206893,
 465: 0.0,
 469: 25.000000000000007,
 502: 0.0,
 504: 0.0,
 544: 11.059907834101383,
 546: 9.523809523809518,
 561: 5.747126436781613,
 587: 0.0,
 635: 0.0,
 651: 11.11111111111111,
 653: 0.0,
 658: 0.0,
 659: 16.666666666666664,
 671: 9.609609609609606,
 692: 0.0,
 712: 0.0,
 719: 17.307692307692307,
 779: 0.0,
 884: 0.0,
 916: 26.76005747126437,
 953: 0.0,
 965: 8.658008658008656,
 9

In [54]:
def MAPE_D(df):
    @njit
    def compute_mape_numba(ratings, mean_ratings):
        return np.mean(np.abs((ratings - mean_ratings) / mean_ratings)) * 100
    
    grouped = df.groupby('recipe_id')
    
    mape_per_recipe = {recipe_id: compute_mape_numba(group['rating'].values, group['mean_rating'].values)
                       for recipe_id, group in grouped}

    return mape_per_recipe

In [58]:
%%time
MAPE_D(reviews_with_mean)

CPU times: total: 6.77 s
Wall time: 6.84 s


{48: 100.0,
 55: 7.894736842105263,
 66: 2.122347066167288,
 91: 7.894736842105263,
 94: 0.0,
 128: 0.0,
 153: 25.806451612903224,
 176: 0.0,
 181: 41.66666666666667,
 186: 7.235142118863049,
 203: 20.377867746288807,
 224: 66.66666666666666,
 240: 16.666666666666664,
 246: 12.444444444444448,
 282: 0.0,
 284: 0.0,
 288: 50.000000000000014,
 289: 0.0,
 314: 6.174957118353343,
 318: 7.894736842105263,
 321: 0.0,
 337: 0.0,
 344: 0.0,
 352: 33.33333333333333,
 360: 0.0,
 373: 0.0,
 378: 8.468468468468473,
 379: 9.523809523809518,
 381: 0.0,
 383: 0.0,
 385: 0.0,
 387: 0.0,
 401: 0.0,
 445: 14.34482758620689,
 465: 0.0,
 469: 25.000000000000007,
 502: 0.0,
 504: 0.0,
 544: 11.059907834101383,
 546: 9.523809523809518,
 561: 5.747126436781613,
 587: 0.0,
 635: 0.0,
 651: 11.11111111111111,
 653: 0.0,
 658: 0.0,
 659: 16.666666666666664,
 671: 9.609609609609606,
 692: 0.0,
 712: 0.0,
 719: 17.30769230769231,
 779: 0.0,
 884: 0.0,
 916: 26.760057471264385,
 953: 0.0,
 965: 8.658008658008656,


In [86]:
# С numba.vectorize и njit

def MAPE_D(df):
    @vectorize([float64(float64, float64)])
    def compute_mape_vectorized(rating, mean_rating):
        return np.abs((rating - mean_rating) / mean_rating) * 100
   
    @njit
    def compute_mape_numba(ratings, mean_ratings):
        return np.mean(compute_mape_vectorized(ratings, mean_ratings))
    
    grouped = df.groupby('recipe_id')
    
    mape_per_recipe = {recipe_id: compute_mape_numba(group['rating'].values, group['mean_rating'].values)
                       for recipe_id, group in grouped}

    return mape_per_recipe

In [87]:
%%time
MAPE_D(reviews_with_mean)

CPU times: total: 6.56 s
Wall time: 6.71 s


{48: 100.0,
 55: 7.894736842105263,
 66: 2.1223470661672867,
 91: 7.894736842105263,
 94: 0.0,
 128: 0.0,
 153: 25.806451612903242,
 176: 0.0,
 181: 41.66666666666667,
 186: 7.235142118863051,
 203: 20.377867746288807,
 224: 66.66666666666666,
 240: 16.66666666666666,
 246: 12.444444444444446,
 282: 0.0,
 284: 0.0,
 288: 50.00000000000001,
 289: 0.0,
 314: 6.174957118353341,
 318: 7.894736842105263,
 321: 0.0,
 337: 0.0,
 344: 0.0,
 352: 33.33333333333333,
 360: 0.0,
 373: 0.0,
 378: 8.46846846846847,
 379: 9.52380952380952,
 381: 0.0,
 383: 0.0,
 385: 0.0,
 387: 0.0,
 401: 0.0,
 445: 14.344827586206897,
 465: 0.0,
 469: 25.000000000000004,
 502: 0.0,
 504: 0.0,
 544: 11.059907834101383,
 546: 9.52380952380952,
 561: 5.747126436781613,
 587: 0.0,
 635: 0.0,
 651: 11.11111111111111,
 653: 0.0,
 658: 0.0,
 659: 16.666666666666664,
 671: 9.609609609609606,
 692: 0.0,
 712: 0.0,
 719: 17.30769230769231,
 779: 0.0,
 884: 0.0,
 916: 26.76005747126437,
 953: 0.0,
 965: 8.658008658008656,
 975

In [90]:
# С numba.vectorize

def MAPE_D(df):
    # @vectorize([float64(float64, float64)])
    def compute_mape_vectorized(rating, mean_rating):
        return np.abs((rating - mean_rating) / mean_rating) * 100

    vectorized_compute_mape = vectorize([float64(float64, float64)])(compute_mape_vectorized)
    grouped = df.groupby('recipe_id')
    
    mape_per_recipe = {recipe_id: vectorized_compute_mape(group['rating'].values, group['mean_rating'].values)
                       for recipe_id, group in grouped}

    return mape_per_recipe

In [91]:
%%time
MAPE_D(reviews_with_mean)

CPU times: total: 6.7 s
Wall time: 6.72 s


{48: array([100.]),
 55: array([ 5.26315789,  5.26315789, 15.78947368,  5.26315789]),
 66: array([ 1.12359551,  1.12359551,  1.12359551,  1.12359551, 19.1011236 ,
         1.12359551,  1.12359551,  1.12359551,  1.12359551,  1.12359551,
         1.12359551,  1.12359551,  1.12359551,  1.12359551,  1.12359551,
         1.12359551,  1.12359551,  1.12359551]),
 91: array([ 5.26315789,  5.26315789, 15.78947368,  5.26315789]),
 94: array([0., 0., 0., 0.]),
 128: array([0., 0.]),
 153: array([27.45098039, 27.45098039, 27.45098039,  1.96078431, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
         1.96078431, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039, 27.45098039, 27.45098039, 27.45098039, 27.45098039,
        27.45098039]),
 176: array([0.]),
 181: array([50. , 62.5, 12.5]),
 186: